In [2]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
from ultralytics import YOLO

In [4]:
model = YOLO('yolov8n-pose.pt')
# Open the video file
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)

In [5]:
def is_t_pose(keypoints):
    try:
        threshold = 20
        if keypoints[5] is not None and keypoints[7] is not None:
            left_shoulder = keypoints[5]
            left_elbow = keypoints[7]
            # Check if the left arm is extended horizontally
            if abs(left_shoulder[1] - left_elbow[1]) < threshold:
                return True

        if keypoints[6] is not None and keypoints[8] is not None:
            right_shoulder = keypoints[6]
            right_elbow = keypoints[8]
            # Check if the right arm is extended horizontally
            if abs(right_shoulder[1] - right_elbow[1]) < threshold:
                return True
    except:
        pass

    return False


In [20]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Pose class and drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
# cap = cv2.videocapture('video.mp4')  # 0 for default webcam
cap = cv2.VideoCapture(0)

# Define the SOS pose detection function
def is_sos_pose(landmarks):
    # Check if arms are raised up (SOS pose: both hands above head)
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

    # Check if wrists are above shoulders (indicating hands up)
    if (left_wrist.y < left_shoulder.y and right_wrist.y < right_shoulder.y):
        return True
    return False

# Use MediaPipe Pose for detecting pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame color to RGB as MediaPipe uses RGB images
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # For performance improvement

        # Process the image to detect pose
        results = pose.process(image)

        # Convert the image color back to BGR for OpenCV
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Check if landmarks are detected
        if results.pose_landmarks:
            # Draw the pose landmarks on the image
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Check if the detected pose matches the SOS pose
            if is_sos_pose(results.pose_landmarks.landmark):
                cv2.putText(image, 'SOS Pose Detected!', (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)

        # Display the resulting image
        cv2.imshow('MediaPipe Pose Estimation', image)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1724522925.310789  202913 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1724522925.410266  217621 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724522925.418422  217625 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/gouravthakur/Desktop/My Folder/coding/Translator/sih24/.venv/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [4]:
cap.release()
cv2.destroyAllWindows()

In [14]:
import cv2
import torch

# Load the YOLOv8 model (you can choose YOLOv8s, YOLOv8m, or YOLOv8l)
model = torch.hub.load('ultralytics/yolov8', 'yolov8s', pretrained=True)

# Initialize webcam
cap = cv2.VideoCapture(0)  # 0 for the default webcam

# Define the SOS pose detection function
def is_sos_pose(detections):
    for detection in detections:
        # Get the bounding box coordinates
        x1, y1, x2, y2, conf, cls = detection
        # Assuming cls 0 corresponds to 'person' in YOLOv8
        if int(cls) == 0:  
            # Check if the hands are raised (y1 < some threshold based on the head position)
            if y1 < 100:  # Adjust this value based on the position of the head in your frame
                return True
    return False

# Start processing frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)
    
    # Get the detections
    detections = results.xyxy[0].numpy()  # Get detections for the first image in the batch

    # Check if SOS pose is detected
    if is_sos_pose(detections):
        cv2.putText(frame, 'SOS Pose Detected!', (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)

    # Draw bounding boxes on detected people
    for x1, y1, x2, y2, conf, cls in detections:
        if int(cls) == 0:  # Only draw boxes for 'person'
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

    # Display the resulting image
    cv2.imshow('YOLOv8 SOS Pose Detection', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


RuntimeError: It looks like there is no internet connection and the repo could not be found in the cache (/Users/gouravthakur/.cache/torch/hub)

In [19]:
import cv2
import numpy as np
from ultralytics import YOLO
import mediapipe as mp

# Load YOLOv8 model (pre-trained or custom)
model = YOLO('yolov8n-pose.pt')  # Replace with actual model path

# Initialize MediaPipe Pose estimator
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Define T-pose criteria
def is_t_pose(landmarks):
    # Check if both arms are extended horizontally
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
    
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    
    # Check if the wrists are horizontally aligned with the shoulders
    left_arm_straight = abs(left_wrist.y - left_shoulder.y) < 0.1 and left_wrist.x < left_shoulder.x < left_elbow.x
    right_arm_straight = abs(right_wrist.y - right_shoulder.y) < 0.1 and right_wrist.x > right_shoulder.x > right_elbow.x
    
    return left_arm_straight and right_arm_straight

# Load video file
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform detection with YOLOv8
    results = model(frame)
    
    # Extract detected persons and perform pose estimation
    for result in results:
        if result['class'] == 'person':
            # Extract the bounding box for the person
            bbox = result['bbox']
            x_min, y_min, x_max, y_max = map(int, bbox)
            person_roi = frame[y_min:y_max, x_min:x_max]
            
            # Convert the image to RGB
            person_rgb = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)
            
            # Perform pose estimation
            pose_results = pose.process(person_rgb)
            
            if pose_results.pose_landmarks:
                landmarks = pose_results.pose_landmarks.landmark
                
                # Check for T-pose
                if is_t_pose(landmarks):
                    # Mark the T-pose in the frame
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    cv2.putText(frame, "T-Pose", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('T-Pose Estimation', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1724522122.333591  202913 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1724522122.431284  209885 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724522122.439711  209886 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


0: 448x640 5 persons, 71.2ms
Speed: 2.3ms preprocess, 71.2ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)


IndexError: too many indices for tensor of dimension 2